<h1>Trying the Multilingual BERT models</h1>

In [1]:
import os
from sentence_transformers import SentenceTransformer, models
from bs4 import BeautifulSoup

/home/erko/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<p>Create tsv data file:</p>

In [2]:
tsv_file = open("en-sv-train.tsv", "w")

with open("en-sv.tmx") as f:
    soup = BeautifulSoup(f, 'xml')
    sentence_pairs = soup.find_all("tu")
    i = 0
    for sp in sentence_pairs:
        sentences = sp.get_text().split("\n")
        en_sentence = sentences[1]
        sv_sentence = sentences[2]
        tsv_file.write(en_sentence+"\t"+sv_sentence+"\n")
        i += 1

<p>Load teacher model:</p>

In [2]:
teacher_model_name = "all-mpnet-base-v2"
# teacher_model_name = "paraphrase-distilroberta-base-v2"
teacher_model = SentenceTransformer(teacher_model_name)

<p>Create student model:</p>

In [3]:
student_model_name = "xlm-roberta-base"
max_seq_length = 256
word_embedding_model = models.Transformer(student_model_name, max_seq_length=max_seq_length)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
student_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<p>Load data from tsv file</p>

In [4]:
from sentence_transformers import SentenceTransformer, LoggingHandler, models, evaluation, losses
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from sentence_transformers.datasets import ParallelSentencesDataset
from datetime import datetime

import os
import logging
import sentence_transformers.util
import csv
import gzip
from tqdm.autonotebook import tqdm
import numpy as np
import zipfile
import io


train_batch_size = 8
train_data = ParallelSentencesDataset(student_model=student_model, teacher_model=teacher_model)
train_data.load_data('en-sv-train.tsv')
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MSELoss(model=student_model)

In [5]:
source_languages = set(['en'])                      # Our teacher model accepts English (en) sentences
target_languages = set(['sv'])    # We want to extend the model to these new languages. For language codes, see the header of the train file

<p>Fit the model:</p>

In [6]:

max_seq_length = 128                #Student model max. lengths for inputs (number of word pieces)
inference_batch_size = 32           #Batch size at inference
max_sentences_per_language = 500000 #Maximum number of  parallel sentences for training
train_max_sentence_length = 250     #Maximum length (characters) for parallel training sentences

num_epochs = 5                       #Train for x epochs
num_warmup_steps = 10000             #Warumup steps

num_evaluation_steps = 1000          #Evaluate performance after every xxxx steps
dev_sentences = 1000                 #Number of parallel sentences to be used for development

import tensorflow

assert tensorflow.version.VERSION.startswith('2.')

gpus = tensorflow.config.experimental.list_physical_devices('GPU')
print(gpus)

#tensorflow.config.experimental.set_virtual_device_configuration(gpus[0], 
#[tensorflow.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

student_model.fit(train_objectives=[(train_dataloader, train_loss)])

[]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]/home/erko/.local/lib/python3.9/site-packages/sentence_transformers/SentenceTransformer.py:547: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  labels = torch.tensor(labels)
Epoch:   0%|          | 0/1 [7:41:26<?, ?it/s]


KeyboardInterrupt: 